In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw2.ipynb")

# HW2: Economic Value to the Consumer (EVC)

In this assignment, we'll explore opportunity costs, sunk costs, Economic Value to the Consumer (EVC), and competitive parity pricing.

In [ ]:
import polars as pl
import numpy as np
from plotnine import ggplot, aes, geom_line, geom_point, geom_hline, labs, theme_minimal, theme

## Part 1: Opportunity Costs & Sunk Costs

A gas station has 30,000 gallons of gas in reserves. The gas in the reserves was purchased from the distributor (including all taxes) for \$2.00 per gallon. The price charged for gas is \$4.00 per gallon. The US Government has disallowed purchases of gas from Russia - with this change the distributor has increased her price to \$3.00. Note you do not anticipate another delivery from the distributor for 3 months.

In [ ]:
# Gas station parameters
gallons_in_reserve = 30000
historical_cost = 2.00  # Original purchase price per gallon
replacement_cost = 3.00  # New distributor price per gallon
current_retail_price = 4.00  # Current price charged to customers

**Question 1:** You are the pricing manager - what cost number do you use to calculate costs: \$2, \$3, or \$4?

*Hint: Think about opportunity costs vs sunk costs. What is the relevant cost for forward-looking decisions?*

In [ ]:
relevant_cost_choice = ...

print(f"The relevant cost for pricing decisions is ${relevant_cost_choice}.00 per gallon.")

In [ ]:
grader.check("q1")

**Question 2:** Say you decide on a price of \$5.50 and sell 10,000 gallons in a month. Calculate your realized profit from this month.


In [ ]:
# Sales parameters
volume_sold = 10000
retail_price = 5.50

In [ ]:
realized_profit = ...

print(f"Realized Profit: ${realized_profit:,.2f}")

In [ ]:
grader.check("q2")

## Part 2: Pricing an Enterprise LLM (EVC)

**Scenario:** You are a Product Manager at **UnclosedAI**, the company behind **TalkTPG**, a revolutionary Large Language Model (LLM). You are in charge of pricing this product for a prospective enterprise client hoping to use it for its sales team. Each license you sell is intended to replace one human sales representative.

The salary and overhead costs for each sales rep is \$60,000/year. The average sales rep generates \$150,000/year in direct profit for the company.

Your goal is to calculate the **Economic Value to the Consumer (EVC)** of a single enterprise TalkTPG license that is intended to replace one human worker.

In [ ]:
# Customer Economics
human_cost = 60000  # Salary + overhead per sales rep
human_profit_generated = 150000  # Profit generated by one human sales rep

**Question 3:** Assume that **TalkTPG** is perfectly equivalent to a human. It generates the exact same profit (\$150,000) as a human sales rep.

Calculate the EVC of one TalkTPG license.

* *Hint: EVC = Reference Value + Differentiation Value.*
* *Reference Value:* The cost the customer avoids by using your product (the price of the alternative).
* *Differentiation Value:* The extra value your product creates (or value lost).


In [ ]:
# TalkTPG Performance
bot_profit_generated = ...

# Reference Value
reference_value_q3 = ...

# Differentiation Value
differentiation_value_q3 = ...

# EVC
evc_q3 = ...

print(f"Reference Value: ${reference_value_q3:,}")
print(f"Differentiation Value: ${differentiation_value_q3:,}")
print(f"Baseline EVC: ${evc_q3:,}")

In [ ]:
grader.check("q3")

**Question 4:** In reality, AI isn't perfect yet. You estimate that TalkTPG is only **80% as effective** as a human in terms of profit generation.

Calculate the **adjusted EVC** for the TalkTPG license.


In [ ]:
# Realistic TalkTPG Performance
bot_profit_generated_real = ...

# Differentiation Value: (Value created by Bot) - (Value created by Human)
differentiation_value_q4 = ...

# EVC = Reference Value + Differentiation Value
# Reference value is still the cost of the alternative (human)
evc_q4 = ...

print(f"Adjusted EVC: ${evc_q4:,}")

In [ ]:
grader.check("q4")

**Question 5:** Suddenly, a competitor, **Humanish**, enters the market. They offer a similar tool, **Clint**, which is also 80% as effective as a human.

Humanish sets their price aggressively at **\$10,000 per license**.

Calculate the updated EVC for TalkTPG given this competitive pressure.


In [ ]:
# Competitor Data
competitor_price = 10000

In [ ]:
# EVC
evc_q5 = ...

print(f"Competitive EVC: ${evc_q5:,}")

In [ ]:
grader.check("q5")

## Part 3: Competitive Parity Pricing

Say there are two sellers on Amazon (**Profnath** and **BordeeBooks**), they both hire Rady students to set value prices for a Pricing Analytics textbook. They are the only two sellers of this book on Amazon. Note that at BN.com (Barnes & Noble), the book is sold for \$5.00. However, the two sellers instruct their new hires to focus solely on the other Amazon seller as a reference product.

The sellers are differentiated as follows:
* **Profnath's Book:** Higher quality rating. Profnath believes his book is worth **\$1.00 more** than Bordee's.
* **Bordee's Book:** Lower quality rating. Bordee believes his book is worth **\$0.25 less** than Profnath's.

Every night at midnight, both sellers run a script to update their prices for the next day. They use Value-Based Pricing logic:
* My Price (Tomorrow) = Competitor's Price (Today) + My Differentiation Value

**Initial State (Day 0):**
* Profnath Price: \$2.00
* Bordee Price: \$1.00

In [ ]:
# Initial prices
initial_price_profnath = 2.00
initial_price_bordee = 1.00

# Perceived differentiation values
profnath_differentiation = 1.00  # Profnath thinks his book is worth $1 more than Bordee's
bordee_differentiation = -0.25   # Bordee thinks his book is worth $0.25 less than Profnath's

In [ ]:
# Pre-written plotting function for the price war simulation
def plot_price_evolution(df, title="Price Evolution Simulation"):
    # Convert wide format to long format for plotnine
    df_long = df.unpivot(
        index='Day',
        on=['Profnath', 'Bordee'],
        variable_name='Seller',
        value_name='Price'
    )
    
    p = (
        ggplot(df_long, aes(x='Day', y='Price', color='Seller'))
        + geom_line(size=1)
        + geom_point(size=2)
        + geom_hline(yintercept=0, linetype='dashed', color='gray', alpha=0.5)
        + labs(title=title, x='Day', y='Price ($)')
        + theme_minimal()
        + theme(figure_size=(10, 6))
    )
    return p

**Question 6:** Write a function `update_prices` that takes the current prices and returns the next day's prices.

The function should implement the pricing logic:
* Profnath's new price = Bordee's current price + \$1.00
* Bordee's new price = Profnath's current price - \$0.25

*Return the new prices as a tuple: `(profnath_tomorrow, bordee_tomorrow)`*

In [ ]:
def update_prices(p_profnath_today, p_bordee_today):
    """
    Calculates tomorrow's prices based on EVC logic.
    
    Profnath thinks: My Value = Bordee_Price + 1.00
    Bordee thinks:   My Value = Profnath_Price - 0.25
    
    Parameters:
    -----------
    p_profnath_today : float
        Profnath's price today
    p_bordee_today : float
        Bordee's price today
        
    Returns:
    --------
    tuple : (profnath_tomorrow, bordee_tomorrow)
    """
    ...
    
    return p_profnath_tomorrow, p_bordee_tomorrow

# Test the function for Day 1
p_prof, p_bord = update_prices(2.00, 1.00)
print(f"Day 1: Profnath ${p_prof:.2f}, Bordee ${p_bord:.2f}")

In [ ]:
grader.check("q6")

**Question 7:** Use your function to simulate the pricing competition for **100 days**. Store the price history in a Polars DataFrame with columns `Day`, `Profnath`, and `Bordee`.

Use the provided `plot_price_evolution` function to visualize the results.

In [ ]:
days = 100
history = {'Day': [], 'Profnath': [], 'Bordee': []}

# Initial Prices
p_prof = initial_price_profnath
p_bord = initial_price_bordee

for day in range(days):
    # Store current prices
    history['Day'].append(day)
    ...
    ...
    
    # Update for next day
    p_prof, p_bord = ...

df_history = pl.DataFrame(history)
print(df_history)

# Plot the price evolution
plot_price_evolution(df_history, title="Price Evolution Simulation (100 Days)")

In [ ]:
grader.check("q7")

<!-- BEGIN QUESTION -->

**Question 8 (open ended):** Look at the price path from Question 7. What is happening to the prices over time? Are they converging or diverging? What is the problem with their pricing analysis?

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Once you have the zip file, upload the **entire** zip file to Gradescope for grading.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)